# Simulation 6
Xinyu HUANG, Qingyuan YAO, Ruohui HU

In [31]:
import threading
import multiprocessing
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import tf_slim as slim
import scipy.signal
from PIL import Image
from PIL import ImageDraw 
from PIL import ImageFont
%matplotlib inline
from helper import *

from random import choice
from time import sleep
from time import time
tf.compat.v1.disable_eager_execution()

In [74]:
gamma = 0.75 # discount rate for advantage estimation and reward discounting
a_size = 2 # Agent can move Left, Right, or Fire
load_model = False
train = True 
model_path = './model_meta'
memory={"1":0,"0":0}

Save_arm -> rs=1
Risky_arm -> rL=4 with p(a_large)=0.125 and 0.5 with 1-0.125
- Agent make 5 forced pulls each of those arms (randomized)
- 20 free pulls

The risky and safe arms could switch from episode to episode with a proba of 0.5

crtic -> LSTM(state_observation, last_reward, last_action, outputting_value_estimate)
actor -> LSTM(state_observation, last_action, reward_prediction_error)

Training : 4x10^6 steps ->6.67x10^4 episodes of 30 trials each.


### ICI, notre tache adapté au two-arm bandit

In [32]:
pb={"high":0.5,"low":0.125}

class TwoArmbandit():
    def __init__(self,proba):
        self.num_actions = 2
        self.state_pos=[0,1]
        self.pb=pb[proba]
        self.reset()
    
    #set the baseline probability of reward for action a.
    #sampling from a uniform Benoulli distribution and held fix for the entire episode
    def reset(self):
        self.timestep = 0 
        self.nb_al = 0
        self.nb_ar = 0
        self.nb = [0,0]
        np.random.shuffle(self.state_pos)
        #np.random.shuffle(self.first_ten)
        self.timestepmax = 30

    ##get action from the network
    def pullArm(self,action,prev_actions):
        self.timestep += 1    
        if self.state_pos[action]==1:
            rvalue=np.random.rand()
            if rvalue<self.pb:
                reward=4
            else:
                reward=0
        else:
            reward=1
        if self.timestep > self.timestepmax: 
            #print("nombre",self.nb_al,self.nb_ar)
            done = True
        else: done = False
        return  reward,done,self.timestep          

In [33]:
class AC_Network():
    def __init__(self,a_size,scope,scope_new,trainer):
        print("in the AC_Network")
        with tf.compat.v1.variable_scope(scope):
            #Input and visual encoding layers
            self.prev_rewards = tf.compat.v1.placeholder(shape=[None,1],dtype=tf.float32)
            self.prev_actions = tf.compat.v1.placeholder(shape=[None],dtype=tf.int32)
            self.timestep = tf.compat.v1.placeholder(shape=[None,1],dtype=tf.float32)
            self.prev_actions_onehot = tf.one_hot(self.prev_actions,a_size,dtype=tf.float32)
            self.state_ob= tf.compat.v1.placeholder(shape=[None,1],dtype=tf.float32)
            print("============")
            print(self.prev_rewards,self.prev_actions_onehot,self.timestep)
            hidden = tf.concat([self.prev_rewards,self.prev_actions_onehot,self.state_ob],1)

            #Recurrent network for temporal dependencies
            #tf.keras.layers.LSTMCell
            lstm_cell = tf.compat.v1.nn.rnn_cell.BasicLSTMCell(48,state_is_tuple=True)
            c_init = np.zeros((1, lstm_cell.state_size.c), np.float32)
            h_init = np.zeros((1, lstm_cell.state_size.h), np.float32)
            self.state_init = [c_init, h_init]
            c_in = tf.compat.v1.placeholder(tf.float32, [1, lstm_cell.state_size.c])
            h_in = tf.compat.v1.placeholder(tf.float32, [1, lstm_cell.state_size.h])
            self.state_in = (c_in, h_in)
            rnn_in = tf.expand_dims(hidden, [0])
            print(rnn_in)
            step_size = tf.shape(self.prev_rewards)[:1]
            state_in = tf.compat.v1.nn.rnn_cell.LSTMStateTuple(c_in, h_in)
            lstm_outputs, lstm_state = tf.compat.v1.nn.dynamic_rnn(
                lstm_cell, rnn_in, initial_state=state_in, sequence_length=step_size,
                time_major=False)
            lstm_c, lstm_h = lstm_state
            self.state_out = (lstm_c[:1, :], lstm_h[:1, :])
            rnn_out = tf.reshape(lstm_outputs, [-1, 48])
            print("rnn_out",rnn_out)
           
            self.actions = tf.compat.v1.placeholder(shape=[None],dtype=tf.int32)
            self.actions_onehot = tf.one_hot(self.actions,a_size,dtype=tf.float32)
            #print("action onehot",self.actions_onehot)
                        
            #Output layers for policy and value estimations
            
            self.value = slim.fully_connected(rnn_out,1,
                activation_fn=None,
                weights_initializer=normalized_columns_initializer(1.0),
                biases_initializer=None)
            #print("policy and value",self.policy,self.value)
            #if scope not in ['global','global_new']:
        #self.value_loss=tf.compat.v1.placeholder(shape=[None],dtype=tf.float32)
        #if scope not in ['global','global_new']:
        #    
        #else:
        #    self.value_loss=100
        with tf.compat.v1.variable_scope(scope_new):
            
            #Only the worker network need ops for loss functions and gradient updating.
            
            self.prev_rewards_new = tf.compat.v1.placeholder(shape=[None,1],dtype=tf.float32)
            self.prev_actions_new = tf.compat.v1.placeholder(shape=[None],dtype=tf.int32)
            self.timestep_new = tf.compat.v1.placeholder(shape=[None,1],dtype=tf.float32)
            self.prev_actions_onehot = tf.one_hot(self.prev_actions_new,a_size,dtype=tf.float32)
            self.state_ob_new= tf.compat.v1.placeholder(shape=[None,1],dtype=tf.float32)
            
            #hidden = tf.concat([self.prev_rewards,self.prev_actions_onehot,self.timestep],1)
            print(self.prev_actions_onehot,self.timestep_new,[[1]])#[[0.5 * tf.reduce_sum(tf.square(self.prev_rewards_new - tf.reshape(self.value,[-1])))]])
            #hidden_new = tf.concat([self.prev_actions_onehot,self.timestep_new,[[0.5 * tf.reduce_sum(tf.square(self.prev_rewards_new[-1] - tf.reshape(self.value,[-1])))]] ],1)
            #hidden_new = tf.concat([self.prev_actions_onehot,[[0.5 * tf.reduce_sum(tf.square(self.prev_rewards_new[-1] - tf.reshape(self.value,[-1])))]],self.state_ob_new ],1)
            
            self.rpe = tf.compat.v1.placeholder(shape=[None,1],dtype=tf.float32)

            hidden_new = tf.concat([self.prev_actions_onehot,self.value,self.state_ob_new ],1)
            lstm_cell_new = tf.compat.v1.nn.rnn_cell.BasicLSTMCell(48,state_is_tuple=True)
            c_init_new = np.zeros((1, lstm_cell.state_size.c), np.float32)
            h_init_new = np.zeros((1, lstm_cell.state_size.h), np.float32)
            self.state_init_new = [c_init_new, h_init_new]
            c_in_new = tf.compat.v1.placeholder(tf.float32, [1, 48])
            h_in_new = tf.compat.v1.placeholder(tf.float32, [1, lstm_cell_new.state_size.h])
            self.state_in_new = (c_in_new, h_in_new)
            rnn_in_new = tf.expand_dims(hidden_new, [0])
            print(rnn_in_new)
            step_size = tf.shape(self.prev_rewards_new)[:1]
            state_in_new = tf.compat.v1.nn.rnn_cell.LSTMStateTuple(c_in_new, h_in_new)
            lstm_outputs, lstm_state = tf.compat.v1.nn.dynamic_rnn(
                lstm_cell_new, rnn_in_new, initial_state=state_in_new, sequence_length=step_size,
                time_major=False)
            lstm_c, lstm_h = lstm_state
            self.state_out_new = (lstm_c[:1, :], lstm_h[:1, :])
            rnn_out_new = tf.reshape(lstm_outputs, [-1, 48])
            print("rnn_out",rnn_out)
            
            self.policy = slim.fully_connected(rnn_out_new,a_size,
                activation_fn=tf.nn.softmax,
                weights_initializer=normalized_columns_initializer(0.01),
                biases_initializer=None)
        if scope_new not in ['global','global_new']:
            
            self.target_v = tf.compat.v1.placeholder(shape=[None],dtype=tf.float32)
            self.advantages = tf.compat.v1.placeholder(shape=[None],dtype=tf.float32)
            self.responsible_outputs = tf.reduce_sum(self.policy * self.actions_onehot, [1])
            #Loss functions
            self.value_loss = 0.5 * tf.reduce_sum(tf.square(self.target_v - tf.reshape(self.value,[-1])))
            self.entropy = - tf.reduce_sum(self.policy * tf.math.log(self.policy + 1e-7))
            self.policy_loss = -tf.reduce_sum(tf.math.log(self.responsible_outputs + 1e-7)*self.advantages)
            self.loss = 0.05* self.value_loss + self.policy_loss - self.entropy * 0.05
            #Get gradients from local network using local losses
            local_vars = tf.compat.v1.get_collection(tf.compat.v1.GraphKeys.TRAINABLE_VARIABLES, scope)
            local_vars1 = tf.compat.v1.get_collection(tf.compat.v1.GraphKeys.TRAINABLE_VARIABLES, scope_new)
            self.gradients = tf.gradients(self.loss,local_vars)
            gradients_new=tf.gradients(self.loss,local_vars1)

            self.var_norms = tf.linalg.global_norm(local_vars)
            grads,self.grad_norms = tf.clip_by_global_norm(self.gradients,50.0)
            grads_new,_ = tf.clip_by_global_norm(gradients_new,50.0)
            #print(self.var_norms) 

            #Apply local gradients to global network
            global_vars = tf.compat.v1.get_collection(tf.compat.v1.GraphKeys.TRAINABLE_VARIABLES, 'global')
            global_vars_new = tf.compat.v1.get_collection(tf.compat.v1.GraphKeys.TRAINABLE_VARIABLES, 'global_new')
            self.apply_grads = trainer.apply_gradients(zip(grads,global_vars))
            trainer.apply_gradients(zip(grads_new,global_vars_new))
            

In [34]:
class Worker():
    def __init__(self,game,name,a_size,trainer,model_path,global_episodes):
        self.name = "worker_" + str(name)
        self.number = name        
        self.model_path = model_path
        self.trainer = trainer
        self.global_episodes = global_episodes
        self.increment = self.global_episodes.assign_add(1)
        self.episode_rewards = []
        self.episode_lengths = []
        self.episode_mean_values = []
        self.summary_writer = tf.compat.v1.summary.FileWriter("train_"+str(self.number))

        #Create the local copy of the network and the tensorflow op to copy global paramters to local network
        print("create a copy")
         
        #print("self.updat_local",self.update_local_ops)   
        self.env = game
        self.a_size=a_size
        self.local_AC = AC_Network(a_size,self.name,self.name+"new",trainer) 
        #self.local_AC_new = AC_Network(a_size,self.name,self.name+"new",trainer) 
        self.update_local_ops = update_target_graph('global',self.name) 
        self.update_local_ops_new = update_target_graph('global_new',self.name+"new")  
        
    def train(self,rollout,sess,gamma,bootstrap_value):
        #print("now train")
        rollout = np.array(rollout)
        #print("rollout",rollout)
        actions = rollout[:,0]
        #print("action",actions)
        rewards = rollout[:,1]
        #print("rewards",rewards)
        timesteps = rollout[:,2]
        #print("timesteps",timesteps)
        prev_rewards = [0] + rewards[:-1].tolist()
        prev_actions = [0] + actions[:-1].tolist()
        values = rollout[:,4]
        self.pr = prev_rewards
        self.pa = prev_actions
        #print("pr,pa",self.pr,self.pa)
        # Here we take the rewards and values from the rollout, and use them to generate the advantage and discounted returns. 
        # The advantage function uses "Generalized Advantage Estimation"
        self.rewards_plus = np.asarray(rewards.tolist() + [bootstrap_value])#np.asarray(rewa=rds.tolist() + [bootstrap_value])
        discounted_rewards = discount(self.rewards_plus,gamma)[:-1]
        #r = self.reward_buffer[t] + self.discount_factor * r
        #discounted_rewards[t] = r
        self.value_plus = np.asarray(values.tolist() + [bootstrap_value])
        advantages = rewards + gamma * self.value_plus[1:] - self.value_plus[:-1]
        advantages = discount(advantages,gamma)

        # Update the global network using gradients from loss
        # Generate network statistics to periodically save
        #print("w10")
        rnn_state = self.local_AC.state_init
        rnn_state_n = self.local_AC.state_init_new
        #print(prev_actions)
        feed_dict = {self.local_AC.target_v:discounted_rewards,
            self.local_AC.prev_rewards:np.vstack(prev_rewards),
            self.local_AC.prev_actions:prev_actions,
            self.local_AC.actions:actions,
            self.local_AC.timestep:np.vstack(timesteps),
            self.local_AC.advantages:advantages,
            self.local_AC.state_in[0]:rnn_state[0],
            self.local_AC.state_in[1]:rnn_state[1],
            self.local_AC.prev_rewards_new:np.vstack(prev_rewards),
            self.local_AC.timestep_new:np.vstack(timesteps),
            #self.local_AC.timestep_new:np.array([[1],[1]]),
            #self.local_AC.prev_actions_new:[[int(i) for i in prev_actions][-1]],
            self.local_AC.prev_actions_new:prev_actions,
            self.local_AC.state_in_new[0]:rnn_state_n[0],
            self.local_AC.state_in_new[1]:rnn_state_n[1],
            self.local_AC.state_ob:[[float(0)] for i in range(len(prev_actions))],
            self.local_AC.state_ob_new:[[float(0)] for i in range(len(prev_actions))],
            self.local_AC.rpe:[[0.5 * np.sum(np.square(np.vstack(prev_rewards) - 1))] for i in range(31)]}
        #print("here")
        #print([[0.5 * np.sum(np.square(np.vstack(prev_rewards) - 1))] for i in range(31)])
        #print(np.vstack(timesteps))
        #print(prev_actions)
        #print("here",self.a_size)
        #aaa=[int(i) for i in prev_actions][-1]
        #print(aaa)
        #zz = tf.one_hot(aaa,self.a_size,dtype=tf.float32)
        #print("wjat")
        #print(zz)
        #print("w11")
        #ttt=tf.cast(tf.constant(prev_actions),dtype=tf.int32)
        #print("w12")
        #print(ttt)
        #tt=tf.one_hot(ttt,self.a_size,dtype=tf.float32)
        #print("here",tt)
        #print(np.vstack(prev_rewards),np.vstack(timesteps))
        #print()
        v_l,p_l,e_l,g_n,l,v_n,apply_grads= sess.run(
            [self.local_AC.value_loss,
            self.local_AC.policy_loss,
            self.local_AC.entropy,
            self.local_AC.grad_norms,
            self.local_AC.loss, 
            self.local_AC.var_norms,
            self.local_AC.apply_grads],
            feed_dict=feed_dict)
        #print("w12")
        return v_l / len(rollout),p_l / len(rollout),e_l / len(rollout),l,g_n,v_n,feed_dict
        #print( "in train",v_l,p_l ,e_l ,l,g_n,v_n,feed_dict)
        #return v_l,p_l ,e_l ,l,g_n,v_n,apply_grads
    def work(self,gamma,global_AC,sess,coord,saver,train,num_episodes=40000):
        print("now work")
        episode_count = sess.run(self.global_episodes)
        #print("episode_count",episode_count)
        total_steps = 0
        print (" Starting worker " + str(self.number))
        spisode_Cl = []
        spisode_Cr = []
        spisode_Rl = []
        spisode_Rr = []
        global memory
        with sess.as_default(), sess.graph.as_default():                 
            while not coord.should_stop() and episode_count <= num_episodes:
                #if i == 10000 : coord.should_stop()
                #print("session run update local ops")
                sess.run(self.update_local_ops)
                sess.run(self.update_local_ops_new)
                #hold = sess.run(self.update_local_ops)
                #print("update from global to local",hold)
                episode_buffer = []
                episode_values = []
                episode_frames = []
                episode_reward = [0,0]
                prev_actions = []
                episode_step_count = 0
                d = False
                r = 0
                a = 0
                t = 0
                self.env.reset()
                rnn_state = self.local_AC.state_init
                rnn_state_n = self.local_AC.state_init_new
                nb_aL = 0
                nb_aR = 0
                nb_rL = 0
                nb_rR = 0
                
                first10=[0,0,0,0,0,1,1,1,1,1]
                np.random.shuffle(first10)

                while d == False :
                    #Take an action using probabilities from policy network output.
                    if t<10:
                        a=first10[t]
                        
                    if r==1:
                        s_ob=0
                        memory["0"]+=1
                    else:
                        s_ob=1
                        memory["1"]+=1
                    # run the critic network
                    v,rnn_state_new = sess.run([self.local_AC.value,self.local_AC.state_out], 
                        feed_dict={
                        self.local_AC.prev_rewards:[[r]],
                        self.local_AC.timestep:[[t]],
                        self.local_AC.prev_actions:[a],
                        self.local_AC.state_in[0]:rnn_state[0],
                        self.local_AC.state_in[1]:rnn_state[1],
                        self.local_AC.state_ob:[[float(s_ob)]]})
                    # run the actor network
                    a_dist,rnn_state_new = sess.run([self.local_AC.policy,self.local_AC.state_out_new], 
                        feed_dict={
                        self.local_AC.prev_rewards:[[r]],
                        self.local_AC.timestep:[[t]],
                        self.local_AC.prev_actions:[a],
                        self.local_AC.state_in[0]:rnn_state[0],
                        self.local_AC.state_in[1]:rnn_state[1],
                        self.local_AC.prev_rewards_new:[[r]],
                        self.local_AC.timestep_new:[[t]],
                        self.local_AC.prev_actions_new:[a],
                        self.local_AC.state_in_new[0]:rnn_state_n[0],
                        self.local_AC.state_in_new[1]:rnn_state_n[1],
                        self.local_AC.state_ob:[[float(s_ob)]],
                        self.local_AC.state_ob_new:[[float(s_ob)]],
                        self.local_AC.rpe:[[0.5 *  np.sum(np.square([[r]] - v))]]})
                    a = np.random.choice(a_dist[0],p=a_dist[0])
  
                    a = np.argmax(a_dist == a)

                    rnn_state = rnn_state_new
                    r,d,t = self.env.pullArm(a,prev_actions) 
                    prev_actions.append(a)

                    if a==0 :
                        #r = 0
                        #Count CL
                        nb_aL+=1 
                        if r==1 : 
                            nb_rL+=1
                        
                    if a==1 : 
                        #r = 1
                        #Count CR
                        nb_aR+=1
                        if r==1:
                            nb_rR+=1  

                   
                    
                    episode_buffer.append([a,r,t,d,v[0,0]])
                    episode_values.append(v[0,0])
                    #episode_frames.append(set_image_bandit(episode_reward,self.env.baseline_prob,a,t))
                    episode_reward[a] += r
                    total_steps += 1
                    episode_step_count += 1
                    #print("episode_step_count",episode_step_count)
                    #print(episode_buffer)
                                            
                self.episode_rewards.append(np.sum(episode_reward))
                #print("self.episode_rewards",self.episode_rewards)
                self.episode_lengths.append(episode_step_count)
                self.episode_mean_values.append(np.mean(episode_values))
                
                # Update the network using the experience buffer at the end of the episode.
                if len(episode_buffer) != 0 and train == True:
                    #print('================================')
                    v_l,p_l,e_l,l,g_n,v_n,apply_grads= self.train(episode_buffer,sess,gamma,0.0)
                    #print("apply_grad",apply_grads)
                    
                # Periodically save gifs of episodes, model parameters, and summary statistics.
                if episode_count != 0:
                    if episode_count % 999 == 0 and self.name == 'worker_0' and train == True:
                        saver.save(sess,self.model_path+'/model-'+str(episode_count)+'.cptk')
                        print ("Saved Model")

                    if episode_count %100 == 0 and self.name == 'worker_0':
                        print(".",episode_count)
                        #self.images = np.array(episode_frames)
                        #make_gif(self.images,'./frames/image'+str(episode_count)+'.gif',
                        #duration=len(self.images)*0.1,true_image=True)

                    mean_reward = np.mean(self.episode_rewards[-50:])
                    mean_length = np.mean(self.episode_lengths[-50:])
                    mean_value = np.mean(self.episode_mean_values[-50:])
                    summary = tf.compat.v1.Summary()
                    summary.value.add(tag='Perf/Reward', simple_value=float(mean_reward))
                    summary.value.add(tag='Perf/Length', simple_value=float(mean_length))
                    summary.value.add(tag='Perf/Value', simple_value=float(mean_value))
                    if train == True:
                        summary.value.add(tag='Losses/Value Loss', simple_value=float(v_l))
                        summary.value.add(tag='Losses/Policy Loss', simple_value=float(p_l))
                        summary.value.add(tag='Losses/Entropy', simple_value=float(e_l))
                        summary.value.add(tag='Losses/Loss', simple_value=float(l))
                        summary.value.add(tag='Losses/Grad Norm', simple_value=float(g_n))
                        summary.value.add(tag='Losses/Var Norm', simple_value=float(v_n))
                    self.summary_writer.add_summary(summary, episode_count)

                    self.summary_writer.flush()
                if self.name == 'worker_0' :
                    sess.run(self.increment)
                    spisode_Cl.append(nb_aL)
                    spisode_Cr.append(nb_aR)
                    spisode_Rl.append(nb_rL)
                    spisode_Rr.append(nb_rR)
                    #print("episode_count",episode_count," : ",spisode_Cl,spisode_Cr,spisode_Rl,spisode_Rr)
                    #print("global_episodes",self.global_episodes,"reward",self.episode_rewards)
                episode_count += 1

## Phase de train

In [80]:
tf.compat.v1.reset_default_graph()
if not os.path.exists(model_path):
    os.makedirs(model_path)
    
if not os.path.exists('./frames'):
    os.makedirs('./frames')

    
with tf.device("/cpu:0"): 
    global_episodes = tf.Variable(0,dtype=tf.int32,name='global_episodes',trainable=False)
    #trainer = tf.train.AdamOptimizer(learning_rate=1e-3)改为tf.optimizers.Adam()
    #trainer = tf.optimizers.Adam(learning_rate=0.001)
    trainer=tf.compat.v1.train.RMSPropOptimizer(learning_rate=0.001)
    master_network = AC_Network(a_size,'global','global_new',None) # Generate global network
    #num_workers = multiprocessing.cpu_count() # Set workers ot number of available CPU threads
    num_workers = 1
    workers = []
    # Create worker classes
    for i in range(num_workers):
        workers.append(Worker(TwoArmbandit("high"),i,a_size,trainer,model_path,global_episodes))
        print("workers",workers)
    saver = tf.compat.v1.train.Saver(max_to_keep=5)

with tf.compat.v1.Session() as sess:
    coord = tf.train.Coordinator()
    if load_model == True:
        print ('Loading Model...')
        ckpt = tf.train.get_checkpoint_state(model_path)
        saver.restore(sess,ckpt.model_checkpoint_path)
    else:
        print("initialise")
        sess.run(tf.compat.v1.global_variables_initializer())
    
    # asynchronous 
    # Start the "work" process for each worker in a separate threat.    
    
    worker_threads = []
    for worker in workers:
        worker_work = lambda: worker.work(gamma,master_network,sess,coord,saver,train)
        thread = threading.Thread(target=(worker_work))
      
        thread.start()
        worker_threads.append(thread)
    coord.join(worker_threads)

in the AC_Network
Tensor("global/Placeholder:0", shape=(None, 1), dtype=float32, device=/device:CPU:0) Tensor("global/one_hot:0", shape=(None, 2), dtype=float32, device=/device:CPU:0) Tensor("global/Placeholder_2:0", shape=(None, 1), dtype=float32, device=/device:CPU:0)
hidden Tensor("global/concat:0", shape=(None, 4), dtype=float32, device=/device:CPU:0)
Tensor("global/ExpandDims:0", shape=(1, None, 4), dtype=float32, device=/device:CPU:0)
rnn_out Tensor("global/Reshape:0", shape=(None, 48), dtype=float32, device=/device:CPU:0)
Tensor("global_new/one_hot:0", shape=(None, 2), dtype=float32, device=/device:CPU:0) Tensor("global_new/Placeholder_2:0", shape=(None, 1), dtype=float32, device=/device:CPU:0) [[1]]
Tensor("global_new/ExpandDims:0", shape=(1, None, 4), dtype=float32, device=/device:CPU:0)


/Users/xinyuhuangmac/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: UserWarning: `tf.nn.rnn_cell.BasicLSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
/Users/xinyuhuangmac/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:81: UserWarning: `tf.nn.rnn_cell.BasicLSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.


rnn_out Tensor("global/Reshape:0", shape=(None, 48), dtype=float32, device=/device:CPU:0)
create a copy
in the AC_Network
Tensor("worker_0/Placeholder:0", shape=(None, 1), dtype=float32, device=/device:CPU:0) Tensor("worker_0/one_hot:0", shape=(None, 2), dtype=float32, device=/device:CPU:0) Tensor("worker_0/Placeholder_2:0", shape=(None, 1), dtype=float32, device=/device:CPU:0)
hidden Tensor("worker_0/concat:0", shape=(None, 4), dtype=float32, device=/device:CPU:0)
Tensor("worker_0/ExpandDims:0", shape=(1, None, 4), dtype=float32, device=/device:CPU:0)
rnn_out Tensor("worker_0/Reshape:0", shape=(None, 48), dtype=float32, device=/device:CPU:0)
Tensor("worker_0new/one_hot:0", shape=(None, 2), dtype=float32, device=/device:CPU:0) Tensor("worker_0new/Placeholder_2:0", shape=(None, 1), dtype=float32, device=/device:CPU:0) [[1]]
Tensor("worker_0new/ExpandDims:0", shape=(1, None, 4), dtype=float32, device=/device:CPU:0)
rnn_out Tensor("worker_0/Reshape:0", shape=(None, 48), dtype=float32, dev

0.5144215011471649 [[ True False]] [[0.50029075 0.49970925]] 0.50029075 [[0.00871558]]
0.5144374160138966 [[False  True]] [[0.5002805  0.49971947]] 0.49971947 [[0.03446081]]
0.51442055584688 [[ True False]] [[0.500263   0.49973702]] 0.500263 [[1.040846]]
0.5144364697014387 [[False  True]] [[0.50025266 0.49974737]] 0.49974737 [[4.159192]]
0.5144196106705119 [[ True False]] [[0.50024366 0.4997563 ]] 0.50024366 [[1.0671189]]
0.5144355235130263 [[False  True]] [[0.50025696 0.49974304]] 0.49974304 [[4.142208]]
0.5144186656180364 [[ True False]] [[0.5002437 0.4997563]] 0.5002437 [[1.0670099]]
0.5144345774486352 [[False  True]] [[0.5002904 0.4997096]] 0.4997096 [[0.01013328]]
0.5144177206894291 [[ True False]] [[0.50024396 0.49975607]] 0.50024396 [[1.0661591]]
0.5144336315082407 [[False  True]] [[0.5002838  0.49971625]] 0.49971625 [[0.02307404]]
0.5144167758846658 [[False  True]] [[0.50024825 0.49975175]] 0.49975175 [[1.050862]]
0.5143999213656171 [[False  True]] [[0.500247   0.49975306]] 0.4

0.5143575838750776 [[ True False]] [[0.50094724 0.49905276]] 0.50094724 [[4.13026]]
0.5143734483209199 [[ True False]] [[0.4999029 0.5000971]] 0.4999029 [[-0.00022061]]
0.5143893117303107 [[ True False]] [[0.4998959  0.50010407]] 0.4998959 [[0.01648288]]
0.5144051741033514 [[ True False]] [[0.4998458 0.5001542]] 0.4998458 [[4.13636]]
0.5144210354401437 [[False  True]] [[0.49984562 0.5001544 ]] 0.5001544 [[4.13679]]
0.5144042330807421 [[False  True]] [[0.5009929 0.4990071]] 0.4990071 [[1.0567447]]
0.5143874318189241 [[False  True]] [[0.5009981  0.49900195]] 0.49900195 [[1.0454313]]
0.5143706316545823 [[ True False]] [[0.5009981  0.49900186]] 0.5009981 [[1.0453544]]
0.5143864920474215 [[False  True]] [[0.499901   0.50009894]] 0.50009894 [[0.00428206]]
0.5143696930111038 [[False  True]] [[0.5009933  0.49900672]] 0.49900672 [[1.0558516]]
0.5143528950720094 [[False  True]] [[0.5009981  0.49900192]] 0.49900192 [[1.0454252]]
0.5143360982300307 [[False  True]] [[0.5009981  0.49900186]] 0.49900

...

0.5140759655282476 [[False  True]] [[0.5008228  0.49917722]] 0.49917722 [[3.968677]]
0.5140595576266318 [[ True False]] [[0.5020008 0.4979992]] 0.5020008 [[0.9712109]]
0.5140750670241286 [[False  True]] [[0.50085604 0.49914405]] 0.49914405 [[3.9519398]]
0.5140586601985128 [[ True False]] [[0.5020012  0.49799877]] 0.5020012 [[0.9710013]]
0.5140741686347099 [[ True False]] [[0.50085604 0.49914405]] 0.50085604 [[3.951938]]
0.5140896760810595 [[False  True]] [[0.50081766 0.49918225]] 0.49918225 [[-0.02876644]]
0.5140732703599694 [[False  True]] [[0.50200075 0.49799928]] 0.49799928 [[0.9712429]]
0.5140568656859303 [[ True False]] [[0.502025   0.49797502]] 0.502025 [[0.9594498]]
0.5140723721998851 [[ True False]] [[0.5008562  0.49914378]] 0.5008562 [[3.9518297]]
0.5140878777242414 [[False  True]] [[0.5008177  0.49918228]] 0.49918228 [[-0.02876771]]
0.5140714741544352 [[ True False]] [[0.50200075 0.49799928]] 0.50200075 [[0.97124285]]
0.5140869787179733 [[ True False]] [[0.50084436 0.49915567

0.5140112599001241 [[ True False]] [[0.50241786 0.49758208]] 0.50241786 [[0.94699556]]
0.5140267175572519 [[False  True]] [[0.5006893 0.4993106]] 0.4993106 [[-0.05453613]]
0.514010368626952 [[ True False]] [[0.5024282  0.49757186]] 0.5024282 [[0.95695066]]
0.5140258253291775 [[ True False]] [[0.5006816  0.49931842]] 0.5006816 [[-0.05092464]]
0.514041281048246 [[False  True]] [[0.5022862  0.49771383]] 0.49771383 [[3.9438944]]
0.5140249332146037 [[False  True]] [[0.50092137 0.49907857]] 0.49907857 [[0.93487006]]
0.5140085864207347 [[ True False]] [[0.50241387 0.49758616]] 0.50241387 [[0.96337116]]
0.5140240412135089 [[False  True]] [[0.5006814 0.4993186]] 0.4993186 [[-0.05084259]]
0.5140076954876459 [[ True False]] [[0.50242805 0.497572  ]] 0.50242805 [[0.9570025]]
0.5140231493258712 [[False  True]] [[0.5006816  0.49931842]] 0.49931842 [[-0.05092398]]
0.514006804667875 [[False  True]] [[0.50242805 0.497572  ]] 0.497572 [[0.9570013]]
0.5139904610492846 [[False  True]] [[0.50245565 0.49754

0.514061955039792 [[ True False]] [[0.50131047 0.49868953]] 0.50131047 [[0.92898035]]
0.5140456563094483 [[ True False]] [[0.50248337 0.49751666]] 0.50248337 [[0.9528859]]
0.5140293586125995 [[False  True]] [[0.5025307  0.49746937]] 0.49746937 [[0.93376034]]
0.5140447657092131 [[False  True]] [[0.5010007  0.49899933]] 0.49899933 [[-0.05814379]]
0.5140601718289319 [[ True False]] [[0.50231797 0.49768195]] 0.50231797 [[3.9217796]]
0.5140438752219122 [[ True False]] [[0.50132304 0.49867702]] 0.50132304 [[0.92360735]]
0.5140275796481217 [[False  True]] [[0.50126517 0.49873486]] 0.49873486 [[0.9483645]]
0.5140429848475242 [[False  True]] [[0.50230855 0.49769148]] 0.49769148 [[3.925526]]
0.5140583890702761 [[ True False]] [[0.5022675  0.49773255]] 0.5022675 [[-0.05824396]]
0.5140420945860277 [[ True False]] [[0.5013223 0.4986777]] 0.5013223 [[0.9239125]]
0.5140258011347428 [[False  True]] [[0.50126517 0.4987349 ]] 0.4987349 [[0.94836956]]
0.514041204437401 [[ True False]] [[0.5021973  0.4978

Exception in thread Thread-28:
Traceback (most recent call last):
  File "/Users/xinyuhuangmac/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/client/session.py", line 1378, in _do_call
    return fn(*args)
  File "/Users/xinyuhuangmac/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/client/session.py", line 1362, in _run_fn
    target_list, run_metadata)
  File "/Users/xinyuhuangmac/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/client/session.py", line 1456, in _call_tf_sessionrun
    run_metadata)
tensorflow.python.framework.errors_impl.CancelledError: Run call was cancelled

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/xinyuhuangmac/opt/anaconda3/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/Users/xinyuhuangmac/opt/anaconda3/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input

KeyboardInterrupt: 

## Phase de test


In [35]:
gamma = 0.75 # discount rate for advantage estimation and reward discounting
a_size = 2 # Agent can move Left, Right, or Fire
load_model = True
train = False 
model_path = './model_meta'
memory={"1":0,"0":0}

In [64]:
tf.compat.v1.reset_default_graph()
if not os.path.exists(model_path):
    os.makedirs(model_path)
    
if not os.path.exists('./frames'):
    os.makedirs('./frames')

    
with tf.device("/cpu:0"): 
    global_episodes = tf.Variable(0,dtype=tf.int32,name='global_episodes',trainable=False)
    #trainer = tf.train.AdamOptimizer(learning_rate=1e-3)改为tf.optimizers.Adam()
    #trainer = tf.optimizers.Adam(learning_rate=0.001)
    trainer=tf.compat.v1.train.RMSPropOptimizer(learning_rate=0.001)
    master_network = AC_Network(a_size,'global','global_new',None) # Generate global network
    #num_workers = multiprocessing.cpu_count() # Set workers ot number of available CPU threads
    num_workers = 1
    workers = []
    # Create worker classes
    for i in range(num_workers):
        workers.append(Worker(TwoArmbandit("high"),i,a_size,trainer,model_path,global_episodes))
        print("workers",workers)
    saver = tf.compat.v1.train.Saver(max_to_keep=5)

with tf.compat.v1.Session() as sess:
    coord = tf.train.Coordinator()
    if load_model == True:
        print ('Loading Model...')
        ckpt = tf.train.get_checkpoint_state(model_path)
        saver.restore(sess,ckpt.model_checkpoint_path)
    else:
        print("initialise")
        sess.run(tf.compat.v1.global_variables_initializer())
    
    # asynchronous 
    # Start the "work" process for each worker in a separate threat.    
    
    worker_threads = []
    for worker in workers:
        worker_work = lambda: worker.work(gamma,master_network,sess,coord,saver,train)
        thread = threading.Thread(target=(worker_work))
      
        thread.start()
        worker_threads.append(thread)
    coord.join(worker_threads)

in the AC_Network
Tensor("global/Placeholder:0", shape=(None, 1), dtype=float32, device=/device:CPU:0) Tensor("global/one_hot:0", shape=(None, 2), dtype=float32, device=/device:CPU:0) Tensor("global/Placeholder_2:0", shape=(None, 1), dtype=float32, device=/device:CPU:0)
hidden Tensor("global/concat:0", shape=(None, 4), dtype=float32, device=/device:CPU:0)
Tensor("global/ExpandDims:0", shape=(1, None, 4), dtype=float32, device=/device:CPU:0)
rnn_out Tensor("global/Reshape:0", shape=(None, 48), dtype=float32, device=/device:CPU:0)
Tensor("global_new/one_hot:0", shape=(None, 2), dtype=float32, device=/device:CPU:0) Tensor("global_new/Placeholder_2:0", shape=(None, 1), dtype=float32, device=/device:CPU:0) [[1]]
Tensor("global_new/ExpandDims:0", shape=(1, None, 4), dtype=float32, device=/device:CPU:0)
rnn_out Tensor("global/Reshape:0", shape=(None, 48), dtype=float32, device=/device:CPU:0)


/Users/xinyuhuangmac/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: UserWarning: `tf.nn.rnn_cell.BasicLSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
/Users/xinyuhuangmac/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:81: UserWarning: `tf.nn.rnn_cell.BasicLSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.


create a copy
in the AC_Network
Tensor("worker_0/Placeholder:0", shape=(None, 1), dtype=float32, device=/device:CPU:0) Tensor("worker_0/one_hot:0", shape=(None, 2), dtype=float32, device=/device:CPU:0) Tensor("worker_0/Placeholder_2:0", shape=(None, 1), dtype=float32, device=/device:CPU:0)
hidden Tensor("worker_0/concat:0", shape=(None, 4), dtype=float32, device=/device:CPU:0)
Tensor("worker_0/ExpandDims:0", shape=(1, None, 4), dtype=float32, device=/device:CPU:0)
rnn_out Tensor("worker_0/Reshape:0", shape=(None, 48), dtype=float32, device=/device:CPU:0)
Tensor("worker_0new/one_hot:0", shape=(None, 2), dtype=float32, device=/device:CPU:0) Tensor("worker_0new/Placeholder_2:0", shape=(None, 1), dtype=float32, device=/device:CPU:0) [[1]]
Tensor("worker_0new/ExpandDims:0", shape=(1, None, 4), dtype=float32, device=/device:CPU:0)
rnn_out Tensor("worker_0/Reshape:0", shape=(None, 48), dtype=float32, device=/device:CPU:0)
workers [<__main__.Worker object at 0x7fce87a1cb90>]
Loading Model...
I

In [65]:
print(memory)

{'1': 7206, '0': 6775}


In [66]:
print(memory["1"]/(memory["1"]+memory["0"]))

0.5154137758386381
